# Europeana API Interactive Examples

This notebook demonstrates how to use the EuropeanaClient library to fetch cultural heritage data from the Europeana API. The examples are based on the scripts in `europeana.py`.

## Setup

Before running the examples, make sure you have:
1. An API key from [Europeana API](https://api.europeana.eu)
2. Set the `EUROPEANA_API_KEY` environment variable or provide it directly in the code
3. Required packages installed (httpx, matplotlib, python-dotenv, europeana-client)

Let's start by importing the necessary libraries and setting up the client.

In [ ]:
# Import required libraries
import collections
import os
from typing import Dict

import matplotlib.pyplot as plt
from dotenv import load_dotenv
from europeana_client import (
    EuropeanaClient,
    SearchFields,
    ProxyFields,
    FilterFields,
    MediaType,
    Profile,
    Reusability,
)

# Load environment variables
load_dotenv()

# Get API key
api_key = os.getenv("EUROPEANA_API_KEY")
if not api_key:
    # If no environment variable is set, you can enter your API key directly here
    api_key = input("Enter your Europeana API key: ")

print(f"API key loaded: {'Yes' if api_key else 'No'}")

In [ ]:
# Helper functions from europeana.py

def analyze_holdings(records: list[Dict]) -> collections.Counter:
    """Analyze records by holding country."""
    
    def get_country(rec: Dict) -> str:
        """Extract country from record, handling various formats."""
        country = rec.get("country")
        if not country:
            return "Unknown"
        elif isinstance(country, list):
            if len(country) > 1:
                # If multiple countries, log a warning and return the first one
                print(f"Multiple countries found for record {rec.get('id')}: {country}")
            return country[0] if country else "Unknown"
        else:
            return country
    
    return collections.Counter(get_country(rec) for rec in records)


def create_visualization(counts: collections.Counter, total_records: int, title_suffix: str = "Indonesian spatial metadata") -> None:
    """Create and display a bar chart of holdings by country."""
    countries, values = zip(*counts.most_common())
    
    plt.figure(figsize=(12, 6))
    plt.barh(countries, values)
    plt.gca().invert_yaxis()
    plt.title(f"Where {total_records} Europeana items with {title_suffix} are held")
    plt.xlabel("Number of items")
    plt.tight_layout()
    plt.show()

print("Helper functions loaded successfully!")

## Example 1: Indonesia Cultural Heritage Search

This example searches for items with Indonesian spatial metadata and analyzes the holdings by country.

In [ ]:
# Create client
client = EuropeanaClient(api_key)

print("=== Indonesia Cultural Heritage Example ===")

# Search parameters
query_text = "Indonesia"
field = ProxyFields.DCTERMS_SPATIAL
max_records = 500  # Reduced for notebook demo

print(f"Searching for items where {field} contains '{query_text}'...")

# Build query
query_builder = client.query().field(field, query_text)

# Fetch records using the iterator
records = []
try:
    for i, record in enumerate(client.search_all(query_builder, max_records=max_records)):
        records.append(record)
        if (i + 1) % 100 == 0:
            print(f"Fetched {i + 1} records...")
except Exception as e:
    print(f"Error during fetch: {e}")
    if not records:
        print("No records retrieved – check query or API key")

print(f"\nTotal records retrieved: {len(records)}")

In [ ]:
# Analyze the data if we have records
if records:
    # Analyze and visualize
    counts = analyze_holdings(records)
    
    # Show top 10 countries
    print("Top 10 holding countries:")
    for country, count in counts.most_common(10):
        print(f"  {country}: {count} items")
    
    # Create visualization
    create_visualization(counts, len(records))
    
    print(f"\nAnalysis complete! Retrieved {len(records)} records from {len(counts)} countries.")
else:
    print("No records to analyze.")

## Example 2: Search by Creator

Search for works by a specific creator, in this case Vincent van Gogh.

In [ ]:
print("=== Creator Search Example ===")
print("Searching for works by Vincent van Gogh...")

query = client.query().creator("Vincent van Gogh").media_type(MediaType.IMAGE)
results = client.search(query)

print(f"Found {results.total_results} items")

if results.items:
    print("\nFirst 5 items:")
    for i, item in enumerate(results.items[:5]):
        title = item.get('title', ['Untitled'])
        title = title[0] if isinstance(title, list) else title
        creator = item.get('dcCreator', ['Unknown creator'])
        creator = creator[0] if isinstance(creator, list) else creator
        print(f"  {i+1}. {title} by {creator}")
        
        # Show additional metadata
        if 'year' in item:
            print(f"      Year: {item['year']}")
        if 'dataProvider' in item:
            provider = item['dataProvider'][0] if isinstance(item['dataProvider'], list) else item['dataProvider']
            print(f"      Provider: {provider}")
        print()

## Example 3: Geographic Search

Search for items from a specific location with geographic coordinates.

In [ ]:
print("=== Geographic Search Example ===")
print("Searching for items from Amsterdam within 25km...")

# Amsterdam coordinates: 52.3676° N, 4.9041° E
query = client.query().location("Amsterdam").geographic(52.3676, 4.9041, 25)
results = client.search(query)

print(f"Found {results.total_results} items")

if results.items:
    print("\nSample items from Amsterdam area:")
    for i, item in enumerate(results.items[:3]):
        title = item.get('title', ['Untitled'])
        title = title[0] if isinstance(title, list) else title
        place = item.get('place', ['Unknown place'])
        place = place[0] if isinstance(place, list) else place
        print(f"  {i+1}. {title}")
        print(f"      Place: {place}")
        print()

## Example 4: Time Period Search

Search for items from a specific time period.

In [ ]:
print("=== Time Period Search Example ===")
print("Searching for items from the 19th century (1800-1899)...")

query = client.query().time_period(1800, 1899).rows(20)
results = client.search(query)

print(f"Found {results.total_results} items")

if results.items:
    print("\nSample 19th century items:")
    for i, item in enumerate(results.items[:5]):
        title = item.get('title', ['Untitled'])
        title = title[0] if isinstance(title, list) else title
        year = item.get('year', ['Unknown year'])
        year = year[0] if isinstance(year, list) else year
        print(f"  {i+1}. {title} ({year})")

## Example 5: Public Domain Content

Search for public domain content with specific subject matter.

In [ ]:
print("=== Public Domain Search Example ===")
print("Searching for public domain landscape images...")

query = (
    client.query()
    .text("landscape")
    .public_domain()
    .media_type(MediaType.IMAGE)
    .rows(20)
)
results = client.search(query)

print(f"Found {results.total_results} public domain items")

if results.items:
    print("\nSample public domain landscape images:")
    for i, item in enumerate(results.items[:5]):
        title = item.get('title', ['Untitled'])
        title = title[0] if isinstance(title, list) else title
        rights = item.get('rights', ['Unknown rights'])
        rights = rights[0] if isinstance(rights, list) else rights
        print(f"  {i+1}. {title}")
        print(f"      Rights: {rights}")
        if 'edmPreview' in item:
            print(f"      Preview: {item['edmPreview']}")
        print()

## Example 6: Institution-Based Search

Search for items from a specific cultural institution.

In [ ]:
print("=== Institution Search Example ===")
print("Searching for items from the British Museum...")

query = client.query().institution("British Museum").rows(20)
results = client.search(query)

print(f"Found {results.total_results} items")

if results.items:
    print("\nSample items from British Museum:")
    for i, item in enumerate(results.items[:5]):
        title = item.get('title', ['Untitled'])
        title = title[0] if isinstance(title, list) else title
        provider = item.get('dataProvider', ['Unknown provider'])
        provider = provider[0] if isinstance(provider, list) else provider
        print(f"  {i+1}. {title}")
        print(f"      Data Provider: {provider}")
        if 'type' in item:
            item_type = item['type']
            item_type = item_type[0] if isinstance(item_type, list) else item_type
            print(f"      Type: {item_type}")
        print()

## Example 7: Faceted Search

Perform a faceted search to understand the distribution of data across different dimensions.

In [ ]:
print("=== Faceted Search Example ===")
print("Getting faceted results for paintings...")

query = (
    client.query()
    .subject("painting")
    .facets(ProxyFields.DC_CREATOR, FilterFields.RIGHTS, FilterFields.PROVIDER)
    .profile(Profile.FACETS)
    .rows(0)  # We don't need the actual items, just the facets
)
results = client.search(query)

print(f"Total paintings found: {results.total_results}")

if results.facets:
    for facet in results.facets:
        print(f"\n{facet['name']} (top 10):")
        for field in facet.get("fields", [])[:10]:
            print(f"  - {field['label']}: {field['count']} items")
else:
    print("No facets returned")

## Example 8: Complex Chained Query

Demonstrate a complex query using multiple chained filters.

In [ ]:
print("=== Complex Chained Query Example ===")
print("Searching for items from Paris between 1880-1890, within 50km of city center...")

# Paris coordinates: 48.8566° N, 2.3522° E
query = (
    client.query()
    .location('Paris')
    .time_period(1880, 1890)
    .public_domain()
    .geographic(48.8566, 2.3522, 50)
    .media_type(MediaType.IMAGE)
    .rows(10)
)

results = client.search(query)
print(f"Found {results.total_results} items matching all criteria")

if results.items:
    print("\nMatching items:")
    for i, item in enumerate(results.items):
        title = item.get('title', ['Untitled'])
        title = title[0] if isinstance(title, list) else title
        creator = item.get('dcCreator', ['Unknown creator'])
        creator = creator[0] if isinstance(creator, list) else creator
        year = item.get('year', ['Unknown year'])
        year = year[0] if isinstance(year, list) else year
        
        print(f"  {i+1}. {title}")
        print(f"      Creator: {creator}")
        print(f"      Year: {year}")
        if 'place' in item:
            place = item['place']
            place = place[0] if isinstance(place, list) else place
            print(f"      Place: {place}")
        print()

## Cleanup

Close the client connection.

In [ ]:
# Close the client
client.close()
print("Client connection closed.")

## Summary

This notebook demonstrated various ways to use the Europeana API:

1. **Basic Search**: Simple text searches with field targeting
2. **Creator Search**: Finding works by specific artists
3. **Geographic Search**: Location-based queries with coordinate filtering
4. **Time Period Search**: Filtering by date ranges
5. **Rights-based Search**: Finding public domain content
6. **Institution Search**: Filtering by data provider
7. **Faceted Search**: Understanding data distribution
8. **Complex Queries**: Chaining multiple filters

Each example shows different aspects of the Europeana Client API and demonstrates how to handle the returned data structure.

### Key API Concepts:

- **Query Builder**: Use `client.query()` to build complex searches
- **Chaining**: Multiple filters can be chained together
- **Media Types**: Filter by IMAGE, TEXT, VIDEO, SOUND, 3D
- **Profiles**: Control the amount of metadata returned
- **Facets**: Analyze data distribution across fields
- **Geographic**: Search within radius of coordinates
- **Rights**: Filter by usage rights (public domain, etc.)

For more information, see:
- [Europeana API Documentation](https://europeana.atlassian.net/wiki/spaces/EF/pages/2385313793/Europeana+APIs+Documentation)
- [Search API Reference](https://api.europeana.eu/console/index.html?url=docs/v3/search.json#/)